In [ ]:
pip install --upgrade astrapy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.3 MB/s eta 0:00:00


In [ ]:
!pip install cassandra-driver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 30.9 MB/s eta 0:00:00


In [ ]:
import json
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

# Load credentials from JSON file
with open("BDTAssignment-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

# Connect to Cassandra
auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cloud_config = {"secure_connect_bundle": "secure-connect-bdtassignment.zip"}
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

# Set keyspace
session.set_keyspace('example')

# Confirm connection
if session:
    print("Connected!")
else:
    print("Connection failed.")


Connected!


Bronze level table

In [ ]:
session.execute("""
CREATE TABLE IF NOT EXISTS bronze_sales (
    region TEXT,
    country TEXT,
    item_type TEXT,
    sales_channel TEXT,
    order_priority TEXT,
    order_date TEXT,
    order_id TEXT PRIMARY KEY,
    ship_date TEXT,
    units_sold INT,
    unit_price FLOAT,
    unit_cost FLOAT,
    total_revenue FLOAT,
    total_cost FLOAT,
    total_profit FLOAT
)
""")

silver level table

In [ ]:
session.execute("""
CREATE TABLE IF NOT EXISTS silver_sales (
    region TEXT,
    country TEXT,
    item_type TEXT,
    sales_channel TEXT,
    order_date TEXT,
    order_id TEXT PRIMARY KEY,
    units_sold INT,
    total_profit FLOAT
)
""")


# golden level table

Profit by Region:

In [ ]:
session.execute("""
CREATE TABLE IF NOT EXISTS gold_profit_by_region (
    region TEXT PRIMARY KEY,
    total_profit FLOAT
)
""")

Units Sold by Item Type:

In [ ]:
session.execute("""
CREATE TABLE IF NOT EXISTS gold_units_by_item (
    item_type TEXT PRIMARY KEY,
    total_units_sold INT
)
""")

Profit by Region and Item Type:

In [ ]:
session.execute("""
CREATE TABLE IF NOT EXISTS gold_profit_by_region_item (
    region TEXT,
    item_type TEXT,
    total_profit FLOAT,
    PRIMARY KEY (region, item_type)
)
""")

#load data

bronze


In [ ]:
import pandas as pd

# Load the CSV data
csv_url = "https://raw.githubusercontent.com/gchandra10/filestorage/main/sales_100.csv"
sales_data = pd.read_csv(csv_url)

# Insert rows into the Bronze table
for _, row in sales_data.iterrows():
    session.execute("""
    INSERT INTO bronze_sales (
        region, country, item_type, sales_channel, order_priority,
        order_date, order_id, ship_date, units_sold, unit_price,
        unit_cost, total_revenue, total_cost, total_profit
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        row['Region'], row['Country'], row['Item Type'], row['Sales Channel'], row['Order Priority'],
        row['Order Date'], str(row['Order ID']), row['Ship Date'], int(row['UnitsSold']),
        float(row['UnitPrice']), float(row['UnitCost']), float(row['TotalRevenue']),
        float(row['TotalCost']), float(row['TotalProfit'])
    ))


silver table

In [ ]:
# Select relevant columns for Silver Table
silver_data = sales_data[['Region', 'Country', 'Item Type', 'Sales Channel',
                          'Order Date', 'Order ID', 'UnitsSold', 'TotalProfit']]

# Insert rows into the Silver table
for _, row in silver_data.iterrows():
    session.execute("""
    INSERT INTO silver_sales (
        region, country, item_type, sales_channel,
        order_date, order_id, units_sold, total_profit
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        row['Region'], row['Country'], row['Item Type'], row['Sales Channel'],
        row['Order Date'], str(row['Order ID']), int(row['UnitsSold']), float(row['TotalProfit'])
    ))


#gold table

table 1

In [ ]:
profit_by_region = sales_data.groupby('Region')['TotalProfit'].sum().reset_index()

for _, row in profit_by_region.iterrows():
    session.execute("""
    INSERT INTO gold_profit_by_region (
        region, total_profit
    ) VALUES (%s, %s)
    """, (row['Region'], float(row['TotalProfit'])))


table 2

In [ ]:
units_by_item = sales_data.groupby('Item Type')['UnitsSold'].sum().reset_index()

for _, row in units_by_item.iterrows():
    session.execute("""
    INSERT INTO gold_units_by_item (
        item_type, total_units_sold
    ) VALUES (%s, %s)
    """, (row['Item Type'], int(row['UnitsSold'])))


table 3

In [ ]:
profit_by_region_item = sales_data.groupby(['Region', 'Item Type']).agg({
    'TotalProfit': 'sum'
}).reset_index()

for _, row in profit_by_region_item.iterrows():
    session.execute("""
    INSERT INTO gold_profit_by_region_item (
        region, item_type, total_profit
    ) VALUES (%s, %s, %s)
    """, (row['Region'], row['Item Type'], float(row['TotalProfit'])))


#verify


bronze table

In [ ]:
rows = session.execute("SELECT * FROM bronze_sales")
for row in rows:
    print(row)

Row(order_id='940980136', country='New Zealand', item_type='Beverages', order_date='10/11/2012', order_priority='M', region='Australia and Oceania', sales_channel='Online', ship_date='11/4/2012', total_cost=184000.515625, total_profit=90640.078125, total_revenue=274640.59375, unit_cost=31.790000915527344, unit_price=47.45000076293945, units_sold=5788)
Row(order_id='363086831', country='Mali', item_type='Household', order_date='8/19/2010', order_priority='M', region='Sub-Saharan Africa', sales_channel='Offline', ship_date='9/7/2010', total_cost=2169465.25, total_profit=715456.4375, total_revenue=2884921.5, unit_cost=502.5400085449219, unit_price=668.27001953125, units_sold=4317)
Row(order_id='176461303', country='Singapore', item_type='Snacks', order_date='1/28/2013', order_priority='C', region='Asia', sales_channel='Online', ship_date='2/7/2013', total_cost=747949.4375, total_profit=423254.625, total_revenue=1171204.125, unit_cost=97.44000244140625, unit_price=152.5800018310547, units_

In [ ]:
rows = session.execute("SELECT * FROM silver_sales")
for row in rows:
    print(row)


Row(order_id='940980136', country='New Zealand', item_type='Beverages', order_date='10/11/2012', region='Australia and Oceania', sales_channel='Online', total_profit=90640.078125, units_sold=5788)
Row(order_id='363086831', country='Mali', item_type='Household', order_date='8/19/2010', region='Sub-Saharan Africa', sales_channel='Offline', total_profit=715456.4375, units_sold=4317)
Row(order_id='176461303', country='Singapore', item_type='Snacks', order_date='1/28/2013', region='Asia', sales_channel='Online', total_profit=423254.625, units_sold=7676)
Row(order_id='438011872', country='Dominica', item_type='Beverages', order_date='6/12/2012', region='Central America and the Caribbean', sales_channel='Online', total_profit=98673.65625, units_sold=6301)
Row(order_id='135178029', country='United Kingdom', item_type='Cosmetics', order_date='5/1/2015', region='Europe', sales_channel='Online', total_profit=180477.0625, units_sold=1038)
Row(order_id='488121116', country='The Bahamas', item_type=

gold table

Profit by Region:

In [ ]:
rows = session.execute("SELECT * FROM gold_profit_by_region")
for row in rows:
    print(row)


Row(region='Australia and Oceania', total_profit=3486940.0)
Row(region='Europe', total_profit=11267281.0)
Row(region='Middle East and North Africa', total_profit=6514261.5)
Row(region='Central America and the Caribbean', total_profit=4252300.0)
Row(region='Asia', total_profit=6749896.0)
Row(region='Sub-Saharan Africa', total_profit=7651892.0)
Row(region='North America', total_profit=1404621.5)


Units Sold by Item Type:

In [ ]:
rows = session.execute("SELECT * FROM gold_units_by_item")
for row in rows:
    print(row)


Row(item_type='Household', total_units_sold=57640)
Row(item_type='Office Supplies', total_units_sold=42814)
Row(item_type='Vegetables', total_units_sold=7368)
Row(item_type='Snacks', total_units_sold=14377)
Row(item_type='Personal Care', total_units_sold=39045)
Row(item_type='Meat', total_units_sold=50437)
Row(item_type='Fruits', total_units_sold=65920)
Row(item_type='Beverages', total_units_sold=45206)
Row(item_type='Cereal', total_units_sold=45776)
Row(item_type='Cosmetics', total_units_sold=65707)
Row(item_type='Baby Food', total_units_sold=20372)
Row(item_type='Clothes', total_units_sold=40148)


Profit by Region and Item Type:

In [ ]:
rows = session.execute("SELECT * FROM gold_profit_by_region_item")
for row in rows:
    print(row)


Row(region='Australia and Oceania', item_type='Baby Food', total_profit=530201.6875)
Row(region='Australia and Oceania', item_type='Beverages', total_profit=90640.078125)
Row(region='Australia and Oceania', item_type='Cereal', total_profit=526224.625)
Row(region='Australia and Oceania', item_type='Clothes', total_profit=667716.5)
Row(region='Australia and Oceania', item_type='Fruits', total_profit=13821.349609375)
Row(region='Australia and Oceania', item_type='Household', total_profit=1637743.875)
Row(region='Australia and Oceania', item_type='Meat', total_profit=20592.0)
Row(region='Europe', item_type='Baby Food', total_profit=992342.75)
Row(region='Europe', item_type='Beverages', total_profit=213727.6875)
Row(region='Europe', item_type='Cereal', total_profit=2322564.0)
Row(region='Europe', item_type='Clothes', total_profit=695109.625)
Row(region='Europe', item_type='Cosmetics', total_profit=2773748.0)
Row(region='Europe', item_type='Fruits', total_profit=32831.4296875)
Row(region='Eu